In [1]:
from MainSrc.PythonVersionHandler import *
from paths import *
from Sparker.SparkLogProcesser.SparkLogOperator import *
from Sparker.SparkLogProcesser.SparkLogReader import *
from Sparker.SparkLogProcesser.SparkLogAnalyzer import *
from Sparker.SparkLogProcesser.SparkLogFileHandler import *
from Sparker.PySparkImports import *
from Sparker.Logic.Trainer import *
from Sparker.Logic.UpdatedProductPreferrer import *

In [6]:
from Sparker.Logic.UpdatedProductPreferrer import *
def getUpdatedLabeledPairsOnJupyter(jour):
    print_('On Jupyter...')
    journeys = jour.filter(lambda x: keyIn('_c', x)).sortBy(lambda x: x['timestamp']).groupBy(lambda x: x['_c']).collect()
    events = []

    def lookActions(action, journey):
        if action['module'] == 'cart' or action['module'] == 'payment':
            journey = modulize(journey)
            id = action['id']
            if keyIn('search', journey):
                for log in journey['search']:
                    if keyIn('ids', log):
                        if id in log['ids']:
                            #events.append({'product': action['id'], 'action': action['module'], 'others': log['ids']})
                            events.append((action['id'], action['module'], log['ids']))

    for c, journey in journeys:
        journey = list(journey)
        for i in range(4, len(journey)):
            lookActions(journey[i], journey[i-4:i])
            
    positive = 1
    negative = -1
    posPairs = []
    negPairs = []
    counts = {}
    labeledPairs = {}

    def posKey(id, module, otherId): return str((id, module, otherId))

    #idsSet = unique(list(map(lambda x: x['products'], events)))
    for id, module, others in events:
        for otherId in others:
            k = posKey(id, module, otherId)
            l = posKey(otherId, module, id)
            if keyIn(k, counts):
                counts[k] += 1
            else:
                counts[k] = 1
            posPairs.append((id, module, otherId, k))
            negPairs.append((otherId, module, id, l))

    for ke in list(counts.keys()):
        id, module, otherId = eval(ke)
        if module == 'payment':
            rev = posKey(otherId, module, id)
            if keyIn(rev, counts):
                if counts[ke] > counts[rev]:
                    labeledPairs[keyPairIds(id, otherId)] = positive
                    labeledPairs[keyPairIds(otherId, id)] = negative
        elif module == 'cart' and not keyIn(posKey(id, 'payment', otherId), counts) and not keyIn(posKey(otherId, 'payment', id), counts):
            rev = posKey(otherId, module, id)
            if keyIn(rev, counts):
                if counts[ke] > counts[rev]:
                    labeledPairs[keyPairIds(id, otherId)] = positive
                    labeledPairs[keyPairIds(otherId, id)] = negative
             
    labeledPairs = sc_().parallelize([(key, v) for key, v in labeledPairs.items()])
    print_(labeledPairs.count(), 'labeled pairs have been generated by', nowStr())
    return labeledPairs

In [7]:
from Sparker.Logic import UpdatedProductPreferrer
original = UpdatedProductPreferrer.getUpdatedLabeledPairs
UpdatedProductPreferrer.getUpdatedLabeledPairs = getUpdatedLabeledPairsOnJupyter

In [1]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
try:
    sc = SparkContext() 
    setSparkContext(sc)
except ValueError:
    pass

In [ ]:
from runGG_Project import main as run
from Sparker.Logic import LogicTests
from Sparker.Logic.TrainDataGenerator import *
def gen(): runUpdatedTrainDataGenerationTest(i = 6)
#run(LogicTests.trainLocalDataTest, False)
run(gen, False)

########################################################################################
########################################################################################
########################################################################################
2017-05-22 00:08:09.888418: Running on osldevptst02...
hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/data/Day1_tv_unitesi_Data/tv_unitesi_all_day_journey has been read as journey by 2017-05-22 00:08:10.960523
